In [ ]:
# IMPORTS
import spacy
import scispacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
import tqdm
import pandas as pd
import re
# import neuralcoref
import networkx as nx
import matplotlib.pyplot as plt
from spacy.lang.en import English

nlp =spacy.load("en_core_web_sm")

In [ ]:
def subtree_matcher(doc):
  subjpass = 0

  for i,tok in enumerate(doc):
    # find dependency tag that contains the text "subjpass"    
    if tok.dep_.find("subjpass") == True:
      subjpass = 1

  x = ''
  y = ''

  # if subjpass == 1 then sentence is passive
  if subjpass == 1:
    for i,tok in enumerate(doc):
      if tok.dep_.find("subjpass") == True:
        y = tok.text

      if tok.dep_.endswith("obj") == True:
        x = tok.text
  
  # if subjpass == 0 then sentence is not passive
  else:
    for i,tok in enumerate(doc):
      if tok.dep_.endswith("subj") == True:
        x = tok.text

      if tok.dep_.endswith("obj") == True:
        y = tok.text

  return x,y

In [ ]:
# Tags I've chosen for relations
deps = ["ROOT", "adj", "attr", "agent", "amod"]

# Tags I've chosen for entities(subjects and objects)
deps = ["compound", "prep", "conj", "mod"]

In [2]:

def sent_(text):
    """
    Function prints out attributes of word from spacy.
    Parameters
    ----------
        text : str
            Block of text with multiple sentences

    Returns
    -------
    None
    Prints attributes of token object.
    text: Get the token text.
    POS: part-of-speech tag.
    DEP: dependency label.
    """
    sent = nlp(text)
    for token in sent:
        # Get the token text, part-of-speech tag and dependency label
        token_text = token.text
        token_pos = token.pos_
        token_dep = token.dep_
        print('{:<12}{:<10}{:<10}{:<10}'.format(token_text, token_pos, token_dep,spacy.explain(token_pos)))

In [ ]:

import spacy
from spacy.lang.en import English
import networkx as nx
import matplotlib.pyplot as plt
# If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.
def getSentences(text):
    nlp = English()
    nlp.add_pipe('sentencizer')
    document = nlp(text)
    return [sent.text for sent in document.sents]

def printToken(token):
    print(token.text, "->", token.dep_)

def appendChunk(original, chunk):
    return original + ' ' + chunk

def isRelationCandidate(token):
    deps = ["ROOT", "adj", "attr", "agent", "amod"]
    return any(subs in token.dep_ for subs in deps)

def isConstructionCandidate(token):
    deps = ["compound", "prep", "conj", "mod"]
    return any(subs in token.dep_ for subs in deps)

def processSubjectObjectPairs(tokens):
    subject = ''
    object = ''
    relation = ''
    subjectConstruction = ''
    objectConstruction = ''
    for token in tokens:
        printToken(token)
        if "punct" in token.dep_:
            continue
        if isRelationCandidate(token):
            relation = appendChunk(relation, token.lemma_)
        if isConstructionCandidate(token):
            if subjectConstruction:
                subjectConstruction = appendChunk(subjectConstruction, token.text)
            if objectConstruction:
                objectConstruction = appendChunk(objectConstruction, token.text)
        if "subj" in token.dep_:
            subject = appendChunk(subject, token.text)
            subject = appendChunk(subjectConstruction, subject)
            subjectConstruction = ''
        if "obj" in token.dep_:
            object = appendChunk(object, token.text)
            object = appendChunk(objectConstruction, object)
            objectConstruction = ''

    print (subject.strip(), ",", relation.strip(), ",", object.strip())
    return (subject.strip(), relation.strip(), object.strip())

def processSentence(sentence):
    tokens = nlp_model(sentence)
    return processSubjectObjectPairs(tokens)

def printGraph(triples):
    G = nx.Graph()
    for triple in triples:
        G.add_node(triple[0])
        G.add_node(triple[1])
        G.add_node(triple[2])
        G.add_edge(triple[0], triple[1])
        G.add_edge(triple[1], triple[2])

    pos = nx.spring_layout(G)
    plt.figure()
    nx.draw(G, pos, edge_color='black', width=1, linewidths=1,
            node_size=500, node_color='seagreen', alpha=0.9,
            labels={node: node for node in G.nodes()})
    plt.axis('off')
    plt.show()

if __name__ == "__main__":

    text = "London is the capital and largest city of England and the United Kingdom. Standing on the River " \
           "Thames in the south-east of England, at the head of its 50-mile (80 km) estuary leading to " \
           "the North Sea, London has been a major settlement for two millennia. " \
           "Londinium was founded by the Romans. The City of London, " \
           "London's ancient core − an area of just 1.12 square miles (2.9 km2) and colloquially known as " \
           "the Square Mile − retains boundaries that follow closely its medieval limits." \
           "The City of Westminster is also an Inner London borough holding city status. " \
           "Greater London is governed by the Mayor of London and the London Assembly." \
           "London is located in the southeast of England." \
           "Westminster is located in London." \
           "London is the biggest city in Britain. London has a population of 7,172,036."

    sentences = getSentences(text)
    nlp_model = spacy.load('en_core_web_sm')

    triples = []
    print (text)
    for sentence in sentences:
        triples.append(processSentence(sentence))

    printGraph(triples)

In [ ]:
subtree_matcher(nlp(candidate_sentences['sentence'][0]))

In [ ]:
path_to_csv = "/Users/awenc/NUIM/CS440/KG_NLPSystem/data/sentences.csv"
candidate_sentences = pd.read_csv(path_to_csv)
print(candidate_sentences.shape)
print(candidate_sentences['sentence'].sample(5))
print(candidate_sentences['sentence'][6])

In [ ]:
# nlp = spacy.load("en_core_sci_sm")

nlp =spacy.load("en_core_web_sm")

In [ ]:
# nlpSci = spacy.load("en_core_sci_sm")
# nlpSci.add_pipe(nlp.create_pipe('sentencizer'))
# doc = nlpSci(text)

from data import *
text = biosci01
doc = nlp(text)
sent01=candidate_sentences['sentence'][0]
sent09=candidate_sentences['sentence'][8]

In [ ]:
sent01
sent09

In [ ]:
# Matcher class object
matcher = Matcher(nlp.vocab)

#define the pattern
pattern = [{'DEP':'amod', 'OP':"?"}, # adjectival modifier
           {'POS':'NOUN'},
           {'LOWER': 'such'},
           {'LOWER': 'as'},
           {'POS': 'PROPN'}]

matcher.add("matching_1", [pattern])
matches = matcher(doc)

span = doc[matches[0][1]:matches[0][2]]
print(span.text)

In [ ]:
from app import clean
for s in candidate_sentences['sentence']:
    print(clean(s))

In [ ]:
def get_relation(sentence):
    """
    Get relation within input sentenceence based on pattern provided.
    params: str - input of single sentenceence.
    """
    doc = nlp(sentence)

    # Matcher class object
    matcher = Matcher(nlp.vocab, validate=True)

    #define the pattern
    # TODO: explore matcher

    """
    Match 0 or more times / match 0 or 1 time(one relation in sencence?)
    Dodałem PROPN ale jeszcze nie przetestowałem.
    """
    # # define such as pattern
    # suchaspattern = [{'POS':'NOUN'}, 
    #        {'LOWER': 'such'}, 
    #        {'LOWER': 'as'}, 
    #        {'POS': 'PROPN'} #proper noun]
    # wrong pattern?
    pattern = [{'DEP':'ROOT'},
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},
            {'POS':'PROPN','OP':'?'},
            {'POS':'ADJ','OP':"?"}]
    #define the pattern
    suchas_pattern = [{'DEP':'amod', 'OP':"?"}, # adjectival modifier
           {'POS':'NOUN'},
           {'LOWER': 'such'},
           {'LOWER': 'as'},
           {'POS': 'PROPN'}]

    andor_pattern = [{'DEP':'amod', 'OP':"?"}, 
           {'POS':'NOUN'}, 
           {'LOWER': 'and', 'OP':"?"}, 
           {'LOWER': 'or', 'OP':"?"}, 
           {'LOWER': 'other'}, 
           {'POS': 'NOUN'}] 

    es_pattern = [{'DEP':'nummod','OP':"?"}, 
           {'DEP':'amod','OP':"?"}, 
           {'POS':'NOUN'}, 
           {'IS_PUNCT':True}, 
           {'LOWER': 'especially'}, 
           {'DEP':'nummod','OP':"?"}, 
           {'DEP':'amod','OP':"?"}, 
           {'POS':'NOUN'}] 
    # TODO Add verb matcher? IDK

    # matcher.add("matching_1", None, pattern) 

    # TODO: obczaic kod.
    # matches = matcher(doc) 
    
    # print(span.text)
    
    # for matcher.add had to change for 2 arguments and second as list
    # it happen after i had to downgrade python version to python=3.6
    # now it gets 2 arguments instead of 3, latter is type list so we can
    # add multiple patterns.
    matcher.add("matching_1", [pattern,suchas_pattern,andor_pattern,es_pattern])

    matches = matcher(doc)
    k = len(matches) - 1

    # span = doc[matches[0][1]:matches[0][2]] 
    span = doc[matches[k][1]:matches[k][2]]

    return(span.text)

get_relation(candidate_sentences['sentence'][4])

In [ ]:
new_sent = candidate_sentences['sentence'][4]+" watermelon such as melon"
get_relation(new_sent)
# new_sent

In [ ]:
# print(get_sents(text)[0])
# print(get_sents(text)[1])
# print(get_sents(text)[2])
# print('end of unclean\n')

# print(clean(get_sents(text)[0].strip()))
# print(clean(get_sents(text)[1].strip()))
# print(clean(get_sents(text)[2].strip()))


In [ ]:
def get_entities(sentence):
    # store entities in variable - object subject
    ent_1 = ''
    ent_2 = ''
    tok_dep = '' # dependency tag of previous token in the sentence
    tok_txt = '' # previous token in the senetence
    pfx = ''
    mod = ''

    for tok in nlp(sentence):
        if tok.dep_ != 'punct':
            if tok.dep_ == 'compound':
                pfx = tok.text
                if tok.dep_ == 'compound':
                    pfx = tok_txt +" "+tok.text

        if tok.dep_.endswith('mod') == True:
            mod = tok.text
            if tok.dep_ == 'compound':
                mod = tok_txt+" "+tok.text

        if tok.dep_.find("subj") == True:
            ent_1 = mod+" "+pfx+" "+tok.text
            tok_txt =''
            tok_dep=''
            pfx = ''
            mod =''

        if tok.dep_.find("obj") == True:
            ent_2 = mod+" "+pfx+" "+tok.text

        tok_dep = tok.dep_
        tok_txt = tok.text
    return [ent_1.strip(), ent_2.strip()]


get_entities(text)

In [ ]:
# get_sent3(text)
# get_sents(text)
# graph(text)
# sent_(text)

In [ ]:
def get_triple(text):

    doc = nlp(text)
    sent = []

    for token in doc:
        # if the token is a verb
        if (token.pos_ in ['VERB','ROOT']):
            phrase =''
            # only extract noun or pronoun subjects
            for sub_tok in token.lefts:
                if (sub_tok.dep_ in ['nsubj','nsubjpass']) and (sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                    # add subject to the phrase
                    phrase += sub_tok.text
                    # save the root of the verb in phrase
                    phrase += ' '+token.lemma_
                    # check for noun or pronoun direct objects
                    for sub_tok in token.rights:
                        # save the object in the phrase
                        if (sub_tok.dep_ in ['dobj']) and (sub_tok.pos_ in ['NOUN','PROPN']):
                            phrase += ' '+sub_tok.text
                            sent.append(phrase)

    return sent
get_triple(fsent) 

In [ ]:
get_entities(clean(text))

In [ ]:
from spacy.matcher import DependencyMatcher

# "[subject] ... initially founded"
pattern = [
  # anchor token: founded
  {
    "RIGHT_ID": "founded",
    "RIGHT_ATTRS": {"ORTH": "founded"}
  },
  # founded -> subject
  {
    "LEFT_ID": "founded",
    "REL_OP": ">",
    "RIGHT_ID": "subject",
    "RIGHT_ATTRS": {"DEP": "nsubj"}
  },
  # "founded" follows "initially"
  {
    "LEFT_ID": "founded",
    "REL_OP": ";",
    "RIGHT_ID": "initially",
    "RIGHT_ATTRS": {"ORTH": "initially"}
  }
]

matcher = DependencyMatcher(nlp.vocab)
matcher.add("FOUNDED", [pattern])
matches = matcher(doc)

In [ ]:
import spacy
from spacy.matcher import DependencyMatcher

# nlp = spacy.load("en_core_web_sm")
matcher = DependencyMatcher(nlp.vocab)
pattern = [
    {
        "RIGHT_ID": "anchor_founded",
        "RIGHT_ATTRS": {"ORTH": "founded"}
    },
    {
        "LEFT_ID": "anchor_founded",
        "REL_OP": ">",
        "RIGHT_ID": "founded_subject",
        "RIGHT_ATTRS": {"DEP": "nsubj"},
    },
    {
        "LEFT_ID": "anchor_founded",
        "REL_OP": ">",
        "RIGHT_ID": "founded_object",
        "RIGHT_ATTRS": {"DEP": "dobj"},
    },
    {
        "LEFT_ID": "founded_object",
        "REL_OP": ">",
        "RIGHT_ID": "founded_object_modifier",
        "RIGHT_ATTRS": {"DEP": {"IN": ["amod", "compound"]}},
    }
]
pattern = [
  {
    "RIGHT_ID": "anchor_founded",       # unique name
    "RIGHT_ATTRS": {"ORTH": "founded"}  # token pattern for "founded"
  },
    
]
matcher.add("FOUNDED", [pattern])
doc = nlp("Smith founded two companies.")
matches = matcher(doc)
print(matches) # [(4851363122962674176, [1])]

In [ ]:
import spacy
from spacy.matcher import DependencyMatcher

nlp = spacy.load("en_core_web_sm")
matcher = DependencyMatcher(nlp.vocab)

pattern = [
    {
        "RIGHT_ID": "anchor_founded",
        "RIGHT_ATTRS": {"ORTH": "founded"}
    },
    {
        "LEFT_ID": "anchor_founded",
        "REL_OP": ">",
        "RIGHT_ID": "founded_subject",
        "RIGHT_ATTRS": {"DEP": "nsubj"},
    },
    {
        "LEFT_ID": "anchor_founded",
        "REL_OP": ">",
        "RIGHT_ID": "founded_object",
        "RIGHT_ATTRS": {"DEP": "dobj"},
    },
    {
        "LEFT_ID": "founded_object",
        "REL_OP": ">",
        "RIGHT_ID": "founded_object_modifier",
        "RIGHT_ATTRS": {"DEP": {"IN": ["amod", "compound"]}},
    }
]

matcher.add("FOUNDED", [pattern])
doc = nlp("Lee, an experienced CEO, has founded two AI startups.")
matches = matcher(doc)

print(matches) # [(4851363122962674176, [6, 0, 10, 9])]
# Each token_id corresponds to one pattern dict
match_id, token_ids = matches[0]
for i in range(len(token_ids)):
    print(pattern[i]["RIGHT_ID"] + ":", doc[token_ids[i]].text)

In [ ]:
ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label": "ORG", "pattern": "MyCorp Inc."}]
ruler.add_patterns(patterns)

doc = nlp("MyCorp Inc. is a company in the U.S.")
print([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
print(list(doc.sents))

# Examine the entities extracted by the mention detector.
# Note that they don't have types like in SpaCy, and they
# are more general (e.g including verbs) - these are any
# spans which might be an entity in UMLS, a large
# biomedical database.
print(doc.ents)

In [ ]:
# We can also visualise dependency parses
# (This renders automatically inside a jupyter notebook!):
from spacy import displacy
displacy.render(next(doc.sents), style='dep', jupyter=True)

# See below for the generated SVG.
# Zoom your browser in a bit!

In [ ]:
for i,_ in enumerate(doc):

    # doc[i].is_sent_start
    print(doc[i].is_sent_start)

In [ ]:
def clean(text):
    # removing new line character
    text = re.sub('\n','', str(text))
    text = re.sub('\n ','',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("- ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing this �, guessing it was apostrophe
    text = re.sub("�s",'',str(text))
    text = re.sub('[a-z]+�','', str(text))
    text = re.sub('�[a-z]+','', str(text))
    text = re.sub('(','',str(text))
    text = re.sub(')','',str(text))
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("- ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    text = re.sub(' +',' ', str(text))

    return text
# type(clean(txt6))
txt6 = clean(txt6)

In [ ]:
txt034="The fox jumped over the log."

In [ ]:
txt6

In [ ]:
sent_(txt6)

In [ ]:
# Get root of sentence
def n_chunk(sent):
    roots = ''
    doc = nlp(sent)
    for chunk in doc.noun_chunks:
        print(f"\nChunk Text: {chunk.text}\n-> Root: {chunk.root.text}\n-> Arc label:{chunk.root.dep_}\n-> Root head: {chunk.root.head.text}\n")
        roots += chunk.root.text+ ' ' +chunk.root.head.text+' '

    return roots

n_chunk(text)


In [ ]:
def noun_chunks_sent(sentence):
    '''
    Take in single sntence and output
    nound chunks
    '''
    chunks = []
    sent = nlp(sentence)
    for chunk in sent.noun_chunks:
#         print(chunk.left())
        chunks.append(chunk)
#         print("Chunk: ",chunk)
    return chunks

In [ ]:
chunks = noun_chunks_sent(txt2)
len(chunks)
chunks




In [ ]:
txt2


In [ ]:
def nnch():
    doc = nlp(txt2)
    # nlp = spacy.load("en_core_web_sm")
    # doc = nlp("bright red apples on the tree")
    print(type(doc))
#     print(len(doc)+" Words added ")
    for i, token in enumerate(doc):
        
        if doc[i].n_lefts != 0:
            if doc[i].n_lefts > 1:
                print(doc[i].n_lefts)
                print([token.text for token in doc[i].lefts])
        
#         print([token.text for token in doc[i].lefts])
#         print(type(doc[i].lefts))
#         print([token.text for token in doc[i].lefts if token !=None])
#         print([token.text for token in doc[i].lefts])  
#         print([token.text for token in doc[i].rights])
#         print(doc[i].n_lefts)  # 2
#         print(doc[i].n_rights)  # 1
#         # TODO Lookup
#         doc[i].n_rights # give int value of how many something is to left or right?
nnch()



In [ ]:
def noun_component(text):
    doc = nlp(text)
    print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
    print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])
    
noun_component(txt2)

    
    

In [3]:
### NA WARSZTAT

def get_relation(sent):
    """
    Get relation within input sentence based on pattern provided.
    params: str - input of single sentence.
    """
    doc = nlp(sent)

    # Matcher class object
    matcher = Matcher(nlp.vocab, validate=True)

    #define the pattern
    # TODO: explore matcher

    """
    Match 0 or more times / match 0 or 1 time(one relation in sencence?)
    Dodałem PROPN ale jeszcze nie przetestowałem.
    """
    pattern = [{'DEP':'ROOT'},
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},
            {'POS':'PROPN','OP':'?'},
            {'POS':'ADJ','OP':"?"}]
    # TODO Add verb matcher?

    matcher.add("matching_1", None, pattern)

    matches = matcher(doc)
    k = len(matches) - 1

    span = doc[matches[k][1]:matches[k][2]]

    return(span.text)

In [ ]:

### WARSZTAT
def get_entities(sentence):
    # store entities in variable - object subject
    ent_1 = ''
    ent_2 = ''
    tok_dep = '' # dependency tag of previous token in the sentence
    tok_txt = '' # previous token in the senetence
    pfx = ''
    mod = ''

    for tok in nlp(sentence):
        if tok.dep_ != 'punct':
            if tok.dep_ == 'compound':
                pfx = tok.text
                if tok.dep_ == 'compound':
                    pfx = tok_txt +" "+tok.text

        if tok.dep_.endswith('mod') == True:
            mod = tok.text
            if tok.dep_ == 'compound':
                mod = tok_txt+" "+tok.text

        if tok.dep_.find("subj") == True:
            ent_1 = mod+" "+pfx+" "+tok.text
            tok_txt =''
            tok_dep=''
            pfx = ''
            mod =''

        if tok.dep_.find("obj") == True:
            ent_2 = mod+" "+pfx+" "+tok.text

        tok_dep = tok.dep_
        tok_txt = tok.text
    return [ent_1.strip(), ent_2.strip()]


In [ ]:
# function: noun(subject), verb, noun(object)
def get_triple(text):
    doc = nlp(text)
    sent = []

    for token in doc:
        # if the token is a verb
        if (token.pos_ in ['VERB','ROOT']):
            phrase =''
            # only extract noun or pronoun subjects
            for sub_tok in token.lefts:
                if (sub_tok.dep_ in ['nsubj','nsubjpass']) and (sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                    # add subject to the phrase
                    phrase += sub_tok.text
                    # save the root of the verb in phrase
                    phrase += ' '+token.lemma_
                    # check for noun or pronoun direct objects
                    for sub_tok in token.rights:
                        # save the object in the phrase
                        if (sub_tok.dep_ in ['dobj']) and (sub_tok.pos_ in ['NOUN','PROPN']):
                            phrase += ' '+sub_tok.text
                            sent.append(phrase)

    return sent

In [ ]:
def noun_component_list(text):
    doc = nlp(text)
    chunks = list(doc.noun_chunks)
    return chunks

In [ ]:
noun_component_list(txt2)

In [ ]:
## For this i have installed python 3.6 and it do not work but, maybe I can make it to work later.
# # Coreference resolution
# import spacy
# import neuralcoref

# nlp = spacy.load('en')
# neuralcoref.add_to_pipe(nlp)
# doc1 = nlp('My sister has a dog. She loves him.')
# print(doc1._.coref_clusters)

# doc2 = nlp('Angela lives in Boston. She is quite happy in that city.')
# for ent in doc2.ents:
#     print(ent._.coref_cluster)

In [ ]:

# Get root of sentence
def n_chunk(sent):
    roots = ''
    doc = nlp(sent)
    for chunk in doc.noun_chunks:
        print(f"\nChunk Text: {chunk.text}\n-> Root: {chunk.root.text}\n-> Arc label:{chunk.root.dep_}\n-> Root head: {chunk.root.head.text}\n")
        roots += chunk.root.text+ ' ' +chunk.root.head.text+' '

    return roots
# print(n_chunk(get_sent(text)[1]))

def noun_chunks_sent(sentence):
    '''
    Take in single sntence and output
    nound chunks
    '''

    sent = nlp(sentence)
    for chunk in sent.noun_chunks:
        print("Chunk: ",chunk)

def noun_component(text):
    doc = nlp(text)
    print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
    print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

def noun_component_list(text):
    doc = nlp(text)
    chunks = list(doc.noun_chunks)
    return chunks